In [ ]:
!pip install diffusers transformers accelerate scipy safetensors

In [ ]:
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionUpscalePipeline
import torch

In [ ]:
model_id = "stabilityai/stable-diffusion-x4-upscaler"
pipeline = StableDiffusionUpscalePipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipeline = pipeline.to("cuda")

In [ ]:
low_res_img = Image.open("/content/tmp.jpg").convert("RGB")
#low_res_img.save("tmp.jpg")
#low_res_img = low_res_img.resize((128, 128))
upscaled_image = pipeline(prompt=prompt, image=low_res_img).images[0]

  0%|          | 0/75 [00:00<?, ?it/s]

In [ ]:
prompt = "white cat, photo"#enlightened advertisement at night, rendered in cinema4d"

In [ ]:
upscaled_image = pipeline(prompt=prompt, image=low_res_img).images[0]

  0%|          | 0/75 [00:00<?, ?it/s]

In [ ]:
upscaled_image

In [ ]:
import cv2
import os

def extract_frames(video_path, output_dir):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video file is opened successfully
    if not cap.isOpened():
        print(f"Error opening video file {video_path}")
        return

    # Create the output directory if it does not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through each frame of the video
    frame_count = 0
    while True:
        # Read the next frame from the video
        ret, frame = cap.read()

        # Check if there are no more frames in the video
        if not ret:
            break

        # Save the frame as an image in the output directory
        frame_path = os.path.join(output_dir, f"frame{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)

        # Increment the frame counter
        frame_count += 1

    # Release the video file
    cap.release()

    print(f"{frame_count} frames extracted from {video_path} and saved to {output_dir}")

In [ ]:
# Example usage
extract_frames("/content/3.mp4", "/content/video0/")

16 frames extracted from /content/3.mp4 and saved to /content/video0/


In [ ]:
for i in range(len(os.listdir("/content/video0/"))):
    low_res_img = Image.open("/content/video0/frame"+str(i)+".jpg").convert("RGB")
    low_res_img = low_res_img.resize((128, 128))
    upscaled_image = pipeline(prompt=prompt, image=low_res_img).images[0]
    upscaled_image.save("/content/video_out/"+str(i)+".jpg")


In [ ]:
import cv2
import os

# Set the input image directory and output video file name
image_dir = "video_out"
output_video = "output_video.avi"

# Get the list of image file names in the directory
image_files = sorted(os.listdir(image_dir))

# Get the first image to set the video dimensions
first_image = cv2.imread(os.path.join(image_dir, image_files[0]))
height, width, channels = first_image.shape

# Define the video codec and frame rate
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 30.0

# Create the video writer object
video_writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Loop through each image file and add it to the video writer object
for image_file in image_files:
    # Read the image file
    image = cv2.imread(os.path.join(image_dir, image_file))

    # Write the image to the video writer object
    video_writer.write(image)

# Release the video writer object and destroy any remaining windows
video_writer.release()
cv2.destroyAllWindows()